In [33]:
import numpy as np
import random
import math

# Define the data
scrubber_inventory = {
    'A1': [1, 1, 1225],
    'A2': [1, 1.5, 1575],
    'A3': [1, 2.5, 2475],
    'A4': [1, 2.5, 1750],
    'A5': [1, 3, 1750],
    'A6': [1, 3.5, 3150],
    'A7': [1, 2.5, 2700],
    'A8': [1, 2.5, 3150],
    'A9': [1, 3.5, 3150],
    'A10': [1, 4, 3825],
    'B1': [1, 1.5, 1400],
    'B2': [1, 2.5, 1720],
    'B3': [1, 3.5, 1720],
    'B4': [1, 3.5, 2200],
    'B5': [1, 5, 2200],
    'C1': [1, 8, 5600],
    'C2': [1, 3, 2970],
    'C3': [1, 3.5, 2460],
    'C4': [1, 4.5, 7740],
    'C5': [1, 5, 9000], 
    'D1': [1, 3, 7100],
    'D2': [1, 6, 4250]
}

scenario1 = {
    'Site 1': [23900, 7],
    'Site 2': [19200, 4],
    'Site 3': [16400, 6],
    'Site 4': [16200, 6],
    'Site 5': [33000, 3]
}

# Define the state representation
# Randomly assign scrubber machines to sites initially
def generate_initial_state(scrubber_inventory, scenario1):
    sites = list(scenario1.keys())
    state = {}
    for site in sites:
        available_scrubbers = list(scrubber_inventory.keys())
        state[site] = random.choice(available_scrubbers)
    return state

# Define the objective function
# Calculate the total cleaning time for all sites
def calculate_total_cleaning_time(state, scrubber_inventory, scenario1):
    total_cleaning_time = 0
    for site in state.keys():
        site_scrubber = state[site]
        site_cleaning_time = scenario1[site][scrubber_inventory[site_scrubber][0]]
        total_cleaning_time += site_cleaning_time
    return total_cleaning_time


# Define the neighborhood structure
# Swap scrubber machines between two sites
def generate_neighboring_state(state):
    sites = list(state.keys())
    site1, site2 = random.sample(sites, 2)
    state_copy = state.copy()
    state_copy[site1], state_copy[site2] = state_copy[site2], state_copy[site1]
    return state_copy


# Define the acceptance probability function
# Accept moves that improve the objective function unconditionally, accept moves that worsen the objective function with a certain probability
def acceptance_probability(delta, temperature):
    return math.exp(-delta / temperature)

# Define the main Simulated Annealing function
def simulated_annealing(scrubber_inventory, scenario1, initial_state, initial_temperature, cooling_rate, iterations):
    current_state = initial_state
    current_cost = calculate_total_cleaning_time(current_state, scrubber_inventory, scenario1)
    best_state = current_state
    best_cost = current_cost
    temperature = initial_temperature

    for i in range(iterations):
        # Generate a neighboring state
        neighboring_state = generate_neighboring_state(current_state)
        neighboring_cost = calculate_total_cleaning_time(neighboring_state, scrubber_inventory, scenario1)

        # Calculate the delta cost
        delta_cost = neighboring_cost - current_cost

        # Accept or reject the neighboring state
        if delta_cost < 0:
            # If the neighboring state is better, accept it
            current_state = neighboring_state
            current_cost = neighboring_cost
            if neighboring_cost < best_cost:
                best_state = neighboring_state
                best_cost = neighboring_cost
        else:
            # If the neighboring state is worse, accept it with a certain probability
            acceptance_prob = acceptance_probability(delta_cost, temperature)
            if random.uniform(0, 1) < acceptance_prob:
                current_state = neighboring_state
                current_cost = neighboring_cost

        # Update the temperature
        temperature *= cooling_rate

    return best_state, best_cost

# Define the hyperparameters
initial_temperature = 1000
cooling_rate = 0.99
iterations = 10000

# Generate an initial state
initial_state = generate_initial_state(scrubber_inventory, scenario1)

# Run the simulated annealing algorithm
best_state, best_cost = simulated_annealing(scrubber_inventory, scenario1, initial_state, initial_temperature, cooling_rate, iterations)

# Extract the scrubbers used in the best state
used_scrubbers_per_site = []
for site in best_state.keys():
    scrubber = best_state[site]
    used_scrubbers_per_site.append(scrubber)

# Print the results
print("Best State:", best_state)
print("Best Cost:", best_cost)
print("Used Scrubbers per Site:", used_scrubbers_per_site)


Best State: {'Site 1': 'B1', 'Site 2': 'A7', 'Site 3': 'C5', 'Site 4': 'A8', 'Site 5': 'B1'}
Best Cost: 26
Used Scrubbers per Site: ['B1', 'A7', 'C5', 'A8', 'B1']
